In [96]:
import numpy as np
from numpy import pi,cos,sin
i = np.array([[1, 0], [0, 1]])
x = np.array([[0, 1], [1, 0]])
y = np.array([[0, -1j], [1j, 0]])
z = np.array([[1, 0], [0, -1]])
cx = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]])
ry = lambda theta: np.array([[cos(theta/2), -sin(theta/2)], [sin(theta/2), cos(theta/2)]])
rx = lambda theta: np.array([[cos(theta/2), -1j*sin(theta/2)], [-1j*sin(theta/2), cos(theta/2)]])
rz = lambda theta: np.array([[np.exp(-1j*theta/2), 0], [0, np.exp(1j*theta/2)]])

In [104]:
alphas = np.random.uniform(0, 2*pi, 100)
# Compare two matrices
comparisons = 0
for alpha in alphas:
    matrix1 = rz(alpha) @ y @ (rz(alpha).conj().T)
    matrix2 = cos(alpha)*y - sin(alpha)*x
    comparison = 1- int(np.allclose(matrix1, matrix2))
    comparisons += comparison
print(f"Matrices are different in {comparisons} cases")


Matrices are different in 0 cases


array([[-0.5      ,  0.8660254],
       [ 0.8660254,  0.5      ]])